In [1]:
"""
seq2seq 모델 문제점
    - 인코더는 LSTM 구조
    - 컨텍스트 벡터 크기 고정(매우 긴 문장 -> 정보손실 큼)
    - 기울기 소실

-> 문제점 개선(attention)
    - 어떤 단어에 더 중점을 둘것인가
    - 역시 인코더는 LSTM 구조
    - 시간이 많이 걸리는 단점이 여전히 있음 => 이를 해결하고자 LSTM끼리의 연결을 해제한 것이 transformer

Bert와 GPT
Bert(Bi-directional encoder representations from transformers)
GPT(generative pre-trained transformer)
문서 분류, 감성 분류, 질의 응답(챗봇), 개체명 인식
    - 개체명 인식 : 삼성 -> 기업이라고 인식

트랜스퍼 러닝(transfer learning:전이학습) : pre-trained, fine-tuning
특정 용도에 맞춰 사전에 학습된(upstream 태스크) 모델을 다른 용도로 재사용(downstream 태스크)할 수 있는 기법

업스트림 태스크(언어 모델) : 다음 단어 예측(GPT), 빈칸 채우기(BERT), 코퍼스의 문맥을 이해하는 문제 -> 학습 -> pre-trained 모델
다운스트림 태스크 : 문서를 분류, 개체명 인식 등
    ex) 독도는 우리 땅, 독도는 경상북도에 속한다, ...
    ex) 독도는 ___ 땅
extrainable AI(설명가능한 AI)
자기지도학습 : 데이터 내에서 X, Y 데이터를 나누어 학습

fine tuning : pre-trained 모델의 가중치를 태스크에 맞춰 모델 전체가 업데이트하는 기법
prompt tuning : pre-trained 모델의 가중치를 태스크에 맞춰 모델 일부만 업데이트하는 기법
in-context learning : 다운스트림 태스크 데이터의 일부만 사용, 모델을 업데이트 하지 않음
    - 퓨샷(few-shot) 러닝 : 다운스트림 태스크 데이터 몇 건만 사용
    - 제로샷(zero-shot) 러닝 : 다운스트림 태스크 데이터 전혀 사용 안함
    - 원샷(one-shot) 러닝 : 다운스트림 태스크 데이터 한 건만 사용

토큰화 : 문자, 단어, 문장, 서브단어
바이트 쌍 인코딩 : 최근 모델 토큰화 기법(GPT)
단어 조각 인코딩(워드피스) : BERT

바이트 쌍 인코딩
    aaabbcaabb (a, b, c)
        - aa 를 x로 치환 (aa가 가장 많이 나오므로)
        - xabbcxbb (bb가 가장 많이 나오므로 y로 다시 치환)
        - xaycxy (x, a, y, c)
    aaabdaaabac (a, b, c, d)
        - XabdXabac
        - XYdXYac
        - ZdZac

    OOV : 단어가 사전에 없는 경우
    low : 5, lower : 2, newest : 6, widest : 3 => l, o, w, e, r, n, s, t, i, d
    ex) lowest 입력 -> OOV

    low => lo w
    lower => lo w e r
    newest => n e w est
    widest => w i d est
    ...
    반복
    ...
    최종 인코딩 결과 => 사전
    l, o, w, e, r, n, s, t, i, d, es, est, lo, low, ne, new, newest, wi, wid, widest

    이제 lowest가 입력되면? OOV가 아님
        - l o w e s t
        - low est => 두 개의 단어로 인코딩이 된다 => OOV가 아니다

워드피스 : 바이트 쌍 인코딩과 비슷, 코퍼스에서 자주 등장한 문자열을 토큰화
    - 차이점은 문자열을 합치는 기준이 다름. 바이트 쌍 인코딩에서는 빈도를 기준으로 합친다.
    - 워드피시는 likelihood가 가장 높은 쌍을 합친다.
    - 코퍼스에 a, b문자에 대해 계산 (#ab : ab문자열의 빈도수, #a : a문자 빈도수)
    - (#ab/n) / ((#a/n) * (#b/n)) 결과값이 큰 값에 해당되는 쌍부터 합치기

센턴스피스 : 문장에서 단어를 분리하는 패키지

허깅페이스 토크나이저 : 자주 등장하는 서브 워드들을 하나의 토큰으로 토큰화

인코더 안에는 LSTM이 들어가있다. 
"""

'\nseq2seq 모델 문제점\n- 컨텍스트 벡터 크기 고정(매우 긴 문장 -> 정보손실 큼)\n- 기울기 소실\n\n-> 문제점 개선(attention)\n- 어떤 단어에 더 중점을 둘것인가\n\nBert와 GPT\nBert(Bi-directional encoder representations from transformers)\nGPT(generative pre-trained transformer)\n문서 분류, 감성 분류, 질의 응답(챗봇), 개체명 인식\n- 개체명 인식 : 삼성 -> 기업이라고 인식\n\n트랜스퍼 러닝(transfer learning) : pre-trained, fine-tuning\n특정 용도에 맞춰 사전에 학습된(upstream 태스크) 모델을 다른 용도로 재사용(downstream 태스크)할 수 있는 기법\n\n업스트림 태스크 : 다음 단어 예측, 빈칸 채우기, 코퍼스의 문맥을 이해하는 문제 -> 학습 -> pre-trained 모델\n다운스트림 태스크 : 문서를 분류, 개체명 인식 등\n'

분류 문제는 보통 BERT를 쓰고   
문장 생성은 보통 GPT를 많이 쓴다

seq2seq : 챗봇, 번역기, 내용 요약, 음성=>텍스트 or 이미지 => CNN => 상황 분류

In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical

In [8]:
df = pd.read_table('../Data/kor.txt', names=['src', 'tar', 'lic'])
df

,src,tar,lic
0,Go.,가.,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
1,Hi.,안녕.,CC-BY 2.0 (France) Attribution: tatoeba.org #5...
2,Run!,뛰어!,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
3,Run.,뛰어.,CC-BY 2.0 (France) Attribution: tatoeba.org #4...
4,Who?,누구?,CC-BY 2.0 (France) Attribution: tatoeba.org #2...
...,...,...,...
5865,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...,CC-BY 2.0 (France) Attribution: tatoeba.org #1...
5866,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...
5867,And the good news is that today the economy is...,"다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, 퇴직...",CC-BY 2.0 (France) Attribution: tatoeba.org #5...
5868,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...,CC-BY 2.0 (France) Attribution: tatoeba.org #9...


In [9]:
# del df['lic']
df = df[['src', 'tar']]
df

,src,tar
0,Go.,가.
1,Hi.,안녕.
2,Run!,뛰어!
3,Run.,뛰어.
4,Who?,누구?
...,...,...
5865,I started a new blog. I'll do my best not to b...,난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되지 ...
5866,I think it's a shame that some foreign languag...,몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 있었...
5867,And the good news is that today the economy is...,"다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, 퇴직..."
5868,If someone who doesn't know your background sa...,만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 원어...


In [11]:
# apply, applymap, filter, lambda, map은 반드시 숙지하도록

In [10]:
df.tar = df.tar.apply(lambda x : '\t'+x+'\n') # sos : \t, eos : \n
df

C:\Users\parkm\AppData\Local\Temp\ipykernel_20920\4232009340.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.tar = df.tar.apply(lambda x : '\t'+x+'\n') # sos : \t, eos : \n


,src,tar
0,Go.,\t가.\n
1,Hi.,\t안녕.\n
2,Run!,\t뛰어!\n
3,Run.,\t뛰어.\n
4,Who?,\t누구?\n
...,...,...
5865,I started a new blog. I'll do my best not to b...,\t난 블로그를 시작했어. 블로그를 초반에만 반짝 많이 하다가 관두는 사람처럼은 되...
5866,I think it's a shame that some foreign languag...,\t몇몇 외국어 선생님이 한 번도 원어민과 공부해본 적도 없으면서 대학을 나올 수 ...
5867,And the good news is that today the economy is...,"\t다음으로 좋은 소식은 오늘 경제가 재성장한다는 것입니다. 임금, 소득, 집값, ..."
5868,If someone who doesn't know your background sa...,\t만일 네 사정도 잘 모르는 사람이 원어민 같다고 말한다면 그건 그 사람이 네가 ...


In [12]:
srcVocab = set()
for line in df.src:
    for c in line:
        srcVocab.add(c)

In [13]:
srcVocab

{' ',
 '!',
 '"',
 '$',
 '%',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '°',
 'ï'}

In [14]:
tarVocab = set()
for line in df.tar:
    for c in line:
        tarVocab.add(c)
tarVocab

{'곰',
 '싫',
 '틀',
 '앉',
 '섬',
 '틱',
 '즐',
 '묵',
 '독',
 '명',
 '벼',
 '보',
 '닮',
 '둬',
 '겐',
 '체',
 '높',
 '뜻',
 '묶',
 '븐',
 '찍',
 '프',
 '벅',
 '준',
 '와',
 '적',
 '투',
 '외',
 '욕',
 '효',
 '겠',
 '무',
 '산',
 '뒤',
 '돕',
 '북',
 '릅',
 '닐',
 '땋',
 '흙',
 '수',
 '임',
 '배',
 '형',
 '삐',
 '맹',
 '의',
 '잎',
 '큐',
 '탄',
 '엎',
 '족',
 '늙',
 '염',
 '딘',
 '석',
 '꾸',
 '텅',
 '젝',
 '헛',
 '생',
 '켰',
 '웹',
 'a',
 '쇠',
 '끊',
 '창',
 '"',
 '표',
 '일',
 '납',
 '앵',
 '상',
 'B',
 '덤',
 '랐',
 '붉',
 'i',
 '찬',
 '꿀',
 '뿔',
 '문',
 '박',
 '뭘',
 '앞',
 '패',
 '한',
 '심',
 '셔',
 '힘',
 '깁',
 '퀴',
 '감',
 '단',
 '끓',
 '낀',
 '례',
 'h',
 '탓',
 '얇',
 '발',
 '금',
 '둘',
 '팀',
 '락',
 '중',
 '탁',
 '빼',
 '사',
 '없',
 '세',
 '눕',
 '윗',
 '약',
 '3',
 '히',
 '빴',
 '젯',
 '딨',
 '톰',
 '붐',
 '닥',
 '녕',
 '튼',
 '쁘',
 '좀',
 '롭',
 '읍',
 '곱',
 '논',
 '쏘',
 '토',
 '줘',
 '왼',
 '직',
 '반',
 '(',
 '꼬',
 '넛',
 '깎',
 '끙',
 '놓',
 '1',
 '굉',
 '하',
 '낙',
 '폰',
 '났',
 '렸',
 '멧',
 '줄',
 '거',
 '꼼',
 '둑',
 '끗',
 '숲',
 '찢',
 '5',
 '네',
 '권',
 '썼',
 '쩔',
 '뻗',
 '타',
 '면',
 '여',
 '채'

In [15]:
len(tarVocab)

1009

In [16]:
len(srcVocab)

74

In [17]:
srcVocab = sorted(list(srcVocab)) # 아스키 코드 값으로 정렬
tarVocab = sorted(list(tarVocab))

In [19]:
srcVocabSize = len(srcVocab) + 1 # 0번 사용하지 않음
tarVocabSize = len(tarVocab) + 1 

In [20]:
tar_to_index = dict([(w, i+1) for i, w in enumerate(tarVocab)])

In [21]:
tar_to_index

{'\t': 1,
 '\n': 2,
 ' ': 3,
 '!': 4,
 '"': 5,
 '%': 6,
 '(': 7,
 ')': 8,
 ',': 9,
 '-': 10,
 '.': 11,
 '/': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '4': 17,
 '5': 18,
 '6': 19,
 '7': 20,
 '8': 21,
 '9': 22,
 ':': 23,
 '?': 24,
 'A': 25,
 'B': 26,
 'C': 27,
 'D': 28,
 'H': 29,
 'I': 30,
 'M': 31,
 'N': 32,
 'R': 33,
 'T': 34,
 'V': 35,
 'a': 36,
 'd': 37,
 'h': 38,
 'i': 39,
 'm': 40,
 'o': 41,
 'p': 42,
 'r': 43,
 't': 44,
 'y': 45,
 '~': 46,
 '°': 47,
 '가': 48,
 '각': 49,
 '간': 50,
 '갇': 51,
 '갈': 52,
 '감': 53,
 '갑': 54,
 '값': 55,
 '갔': 56,
 '강': 57,
 '갖': 58,
 '같': 59,
 '갛': 60,
 '개': 61,
 '객': 62,
 '갰': 63,
 '걀': 64,
 '걔': 65,
 '거': 66,
 '걱': 67,
 '건': 68,
 '걷': 69,
 '걸': 70,
 '검': 71,
 '겁': 72,
 '것': 73,
 '겉': 74,
 '게': 75,
 '겐': 76,
 '겠': 77,
 '겨': 78,
 '격': 79,
 '겪': 80,
 '견': 81,
 '결': 82,
 '겼': 83,
 '경': 84,
 '곁': 85,
 '계': 86,
 '고': 87,
 '곡': 88,
 '곤': 89,
 '곧': 90,
 '골': 91,
 '곰': 92,
 '곱': 93,
 '곳': 94,
 '공': 95,
 '과': 96,
 '관': 97,
 '광': 98,
 '괜': 99,
 '괴': 100,
 '굉': 

In [22]:
src_to_index=dict([(w, i+1) for i, w in enumerate(srcVocab)])
src_to_index

{' ': 1,
 '!': 2,
 '"': 3,
 '$': 4,
 '%': 5,
 "'": 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 ';': 21,
 '?': 22,
 'A': 23,
 'B': 24,
 'C': 25,
 'D': 26,
 'E': 27,
 'F': 28,
 'G': 29,
 'H': 30,
 'I': 31,
 'J': 32,
 'K': 33,
 'L': 34,
 'M': 35,
 'N': 36,
 'O': 37,
 'P': 38,
 'Q': 39,
 'R': 40,
 'S': 41,
 'T': 42,
 'U': 43,
 'V': 44,
 'W': 45,
 'Y': 46,
 'a': 47,
 'b': 48,
 'c': 49,
 'd': 50,
 'e': 51,
 'f': 52,
 'g': 53,
 'h': 54,
 'i': 55,
 'j': 56,
 'k': 57,
 'l': 58,
 'm': 59,
 'n': 60,
 'o': 61,
 'p': 62,
 'q': 63,
 'r': 64,
 's': 65,
 't': 66,
 'u': 67,
 'v': 68,
 'w': 69,
 'x': 70,
 'y': 71,
 'z': 72,
 '°': 73,
 'ï': 74}

In [23]:
enc_input = []
for line in df.src:
    enc_line = []
    for c in line:
        enc_line.append(src_to_index[c])
    enc_input.append(enc_line)

In [24]:
print(enc_input)

[[29, 61, 9], [30, 55, 9], [40, 67, 60, 2], [40, 67, 60, 9], [45, 54, 61, 22], [45, 61, 69, 2], [26, 67, 49, 57, 2], [28, 55, 64, 51, 2], [30, 51, 58, 62, 2], [30, 55, 50, 51, 9], [32, 67, 59, 62, 2], [32, 67, 59, 62, 9], [41, 66, 47, 71, 9], [45, 47, 55, 66, 2], [45, 47, 55, 66, 2], [45, 47, 55, 66, 9], [24, 51, 53, 55, 60, 9], [30, 51, 58, 58, 61, 2], [30, 51, 58, 58, 61, 9], [31, 1, 65, 51, 51, 9], [31, 1, 66, 64, 71, 9], [31, 1, 69, 61, 60, 2], [37, 54, 1, 60, 61, 2], [40, 51, 58, 47, 70, 9], [41, 54, 61, 61, 66, 2], [41, 59, 55, 58, 51, 9], [41, 61, 64, 64, 71, 22], [41, 61, 64, 64, 71, 22], [41, 61, 64, 64, 71, 22], [23, 66, 66, 47, 49, 57, 2], [23, 66, 66, 47, 49, 57, 2], [28, 64, 51, 51, 72, 51, 2], [29, 51, 66, 1, 67, 62, 9], [29, 61, 66, 1, 55, 66, 2], [29, 61, 66, 1, 55, 66, 22], [30, 51, 1, 64, 47, 60, 9], [30, 67, 53, 1, 59, 51, 9], [31, 1, 57, 60, 61, 69, 9], [31, 1, 58, 61, 65, 66, 9], [31, 1, 63, 67, 55, 66, 9], [31, 1, 69, 61, 64, 57, 9], [34, 55, 65, 66, 51, 60, 9], [

In [27]:
dec_input = []
for line in df.tar:
    dec_line = []
    for c in line:
        dec_line.append(tar_to_index[c])
    dec_input.append(dec_line)

In [28]:
dec_input

[[1, 48, 11, 2],
 [1, 607, 214, 11, 2],
 [1, 319, 629, 4, 2],
 [1, 319, 629, 11, 2],
 [1, 225, 103, 24, 2],
 [1, 672, 663, 4, 2],
 [1, 560, 644, 4, 2],
 [1, 593, 4, 2],
 [1, 268, 663, 753, 4, 2],
 [1, 564, 629, 11, 2],
 [1, 728, 938, 4, 2],
 [1, 728, 938, 952, 11, 2],
 [1, 48, 388, 3, 707, 629, 11, 2],
 [1, 127, 245, 354, 4, 2],
 [1, 716, 135, 4, 2],
 [1, 127, 245, 354, 11, 2],
 [1, 575, 712, 952, 11, 2],
 [1, 607, 214, 4, 2],
 [1, 607, 214, 11, 2],
 [1, 610, 615, 629, 11, 2],
 [1, 575, 268, 952, 476, 75, 11, 2],
 [1, 191, 48, 3, 698, 83, 629, 4, 2],
 [1, 605, 239, 3, 698, 344, 4, 2],
 [1, 763, 730, 952, 11, 2],
 [1, 593, 4, 2],
 [1, 676, 629, 11, 2],
 [1, 746, 557, 950, 239, 245, 388, 24, 2],
 [1, 715, 3, 423, 3, 294, 638, 629, 669, 11, 2],
 [1, 433, 326, 87, 3, 945, 552, 178, 669, 24, 2],
 [1, 95, 79, 4, 2],
 [1, 95, 79, 952, 4, 2],
 [1, 152, 770, 386, 4, 2],
 [1, 701, 629, 178, 11, 2],
 [1, 610, 77, 629, 4, 2],
 [1, 610, 77, 629, 24, 2],
 [1, 119, 234, 3, 250, 358, 245, 11, 2],
 [1,

In [29]:
dec_tar = []
for line in df.tar:
    enc_line = []
    t = 0
    for c in line:
        if t > 0: # 맨 앞의 문자 제거
            enc_line.append(tar_to_index[c])
        t += 1
    dec_tar.append(enc_line)

dec_tar

[[48, 11, 2],
 [607, 214, 11, 2],
 [319, 629, 4, 2],
 [319, 629, 11, 2],
 [225, 103, 24, 2],
 [672, 663, 4, 2],
 [560, 644, 4, 2],
 [593, 4, 2],
 [268, 663, 753, 4, 2],
 [564, 629, 11, 2],
 [728, 938, 4, 2],
 [728, 938, 952, 11, 2],
 [48, 388, 3, 707, 629, 11, 2],
 [127, 245, 354, 4, 2],
 [716, 135, 4, 2],
 [127, 245, 354, 11, 2],
 [575, 712, 952, 11, 2],
 [607, 214, 4, 2],
 [607, 214, 11, 2],
 [610, 615, 629, 11, 2],
 [575, 268, 952, 476, 75, 11, 2],
 [191, 48, 3, 698, 83, 629, 4, 2],
 [605, 239, 3, 698, 344, 4, 2],
 [763, 730, 952, 11, 2],
 [593, 4, 2],
 [676, 629, 11, 2],
 [746, 557, 950, 239, 245, 388, 24, 2],
 [715, 3, 423, 3, 294, 638, 629, 669, 11, 2],
 [433, 326, 87, 3, 945, 552, 178, 669, 24, 2],
 [95, 79, 4, 2],
 [95, 79, 952, 4, 2],
 [152, 770, 386, 4, 2],
 [701, 629, 178, 11, 2],
 [610, 77, 629, 4, 2],
 [610, 77, 629, 24, 2],
 [119, 234, 3, 250, 358, 245, 11, 2],
 [607, 605, 753, 11, 2],
 [610, 605, 11, 2],
 [178, 234, 3, 703, 629, 458, 358, 245, 11, 2],
 [178, 3, 178, 50, 

In [33]:
max_src_len = max([len(line) for line in df.src]) # src 문장의 최대 길이
max_src_len

537

In [34]:
max_tar_len = max([len(line) for line in df.tar]) # tar 문장의 최대 길이
max_tar_len

298

In [37]:
enc_input = pad_sequences(enc_input, maxlen=max_src_len)
dec_input = pad_sequences(dec_input, maxlen=max_tar_len)
dec_tar = pad_sequences(dec_tar, maxlen=max_tar_len, padding='pre')

In [38]:
enc_input = to_categorical(enc_input)
dec_input = to_categorical(dec_input)
dec_tar = to_categorical(dec_tar)

In [40]:
enc_input.shape # 인코더 안에 LSTM셀이 537개. 각 단어는 75차원

(5870, 537, 75)

In [42]:
dec_input.shape 

(5870, 298, 1010)

In [43]:
from tensorflow.keras.layers import Embedding, Dense, LSTM, Input
from tensorflow.keras.models import Model
import numpy as np

**functional API**   
Input(shape = (None, srcVocabSize)) => 입력 문장의 개수는 정해지지 않음, 차원은 srcVocabSize이다.

In [44]:
# 인코더
enc_inputs = Input(shape=(None, srcVocabSize))
enc_lstm = LSTM(units=256, return_state=True)
enc_outputs, stateH, stateC = enc_lstm(enc_inputs)
enc_stats = [stateH, stateC] # context vector(마지막 셀의 출력)
# enc_outputs, stateH, stateC = LSTM(units=256, return_state=True) # 256차원으로 출력
# stateH : 은닉상태정보, stateC : 셀의 상태 정보

In [46]:
# 디코더
dec_inputs = Input(shape=(None, tarVocabSize))
dec_lstm = LSTM(units=256, return_state=True, return_sequences=True)

# 인코더 => 정보 => 디코더
dec_outputs, _, _ = dec_lstm(dec_inputs, initial_state=enc_stats) # 상태는 필요 없기 때문에 버림
dec_layer = Dense(tarVocabSize, activation='softmax')
dec_layer(dec_outputs)

<KerasTensor: shape=(None, None, 1010) dtype=float32 (created by layer 'dense')>

In [53]:
model = Model([enc_inputs, dec_inputs, dec_outputs])

In [54]:
model.compile(optimizer='adam', loss='categorical_crossentropy')

In [55]:
model.fit(x=[enc_input, dec_input], y=dec_tar, batch_size=64, epochs=10, validation_split=0.2)

MemoryError: Unable to allocate 5.27 GiB for an array with shape (4696, 298, 1010) and data type float32

In [56]:
dec_input.shape

(5870, 298, 1010)

In [58]:
enc_input.shape

(5870, 537, 75)

In [61]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

In [63]:
dec_outputs, state_h, state_c = dec_lstm(dec_inputs, initial_state=decoder_states_inputs)

In [64]:
decoder_states = [state_h, state_c]
dec_outputs = dec_layer(dec_outputs) # softmax
decoder_model = Model(inputs=[dec_inputs]+decoder_states_inputs, outputs=[dec_outputs]+decoder_states)

입력문장 : 영문   
정답문장 : 한글   
예측번역문장 : 한글

In [65]:
def dec_sequence(input_seq):
    pass

In [60]:
input_seq = df.src[3]
dec_sentence = dec_sequence(input_seq)